# Day 01 - Advent of Code 2018

https://adventofcode.com/2018/

## Day 1a

Problem: Add up strings that represent values, for example, 

- +1, +1, +1 results in  3
- +1, +1, -2 results in  0
- -1, -2, -3 results in -6

My input file is at `day1a.input`. 

In [5]:
;head day1a.input

+6
-17
+16
+7
+12
+2
-7
-5
-4
-16


In [6]:
;tail day1a.input

+11
-35
-23
+11
-10
-5
+3
-15
-15
+128514


In [9]:
;wc -l day1a.input

    1014 day1a.input


In [10]:
function day1a(filename::AbstractString, startValue::Int64 = 0)
    value = startValue
    for line in eachline(filename)
        value += parse(Int64, line)
    end
    return value
end 

day1a (generic function with 2 methods)

In [11]:
day1a("day1a.input")

470

From slack - in one line! -

In [139]:
sum( parse.(Int, readlines("day1a.input")))

470

## Day 1b

Loop through the values and watch the frequency. If we make a frequency that we've seen before, then stop and report that frequency. We may need to loop through the list more than once to get there. For example, 

```+7, +7, -2, -7, -4 first reaches 14 twice.``` Let's try this example

In [60]:
l = [7, 7, -2, -7, -4]   # We put a zero at the front for the previous sum

5-element Array{Int64,1}:
  7
  7
 -2
 -7
 -4

In [61]:
function day1b(l::Array{Int, 1})
    vals::Array{Int64, 1} = [0]
    rounds = 1
    while true
        for e in l
            newVal = vals[end] + e
            if newVal in vals
                return newVal, rounds
            end
            append!(vals, newVal)
        end
        rounds += 1
    end
end

day1b (generic function with 1 method)

In [62]:
day1b(l)

(14, 3)

So, for ours, let's turn the file into an integer array

In [145]:
ld = [parse(Int, e) for e in eachline("day1a.input")]

1014-element Array{Int64,1}:
      6
    -17
     16
      7
     12
      2
     -7
     -5
     -4
    -16
      2
     12
    -16
      ⋮
     12
      5
     11
    -35
    -23
     11
    -10
     -5
      3
    -15
    -15
 128514

In [150]:
@time day1b(ld)

  3.424009 seconds (23 allocations: 3.001 MiB)


(790, 138)

So the repeated value is 790. We had to run through the list of values 138 times!

## From Slack

From Slack... here's a slow way...

In [172]:
function day1bs1(input::Array{Int, 1})  # From Nick Robinson
    total = 0
    seen = Set{Int}([0])  # This makes the search fast?
    for i in Iterators.cycle(input) # Neat!  Just keeps looping over array, entry value into i
        total += i
        if total in seen
            return(total)
            break
        else
            push!(seen, total)
        end
    end
end

day1bs1 (generic function with 1 method)

In [2]:
@time day1bs1(ld)

UndefVarError: UndefVarError: day1bs1 not defined

Here's an even faster way

In [159]:
function day1bs2(input)  # From Dave F Kleinschmidt
    seen = BitSet([0])
    freq = 0
    for i in Iterators.cycle(input)
        freq += i
        if freq ∈ seen
            return freq
        else
            push!(seen, freq)
        end
    end
end

day1bs2 (generic function with 1 method)

In [164]:
@time day1bs2(ld)

  0.000411 seconds (14 allocations: 49.094 KiB)


790

What about using a regular array?

In [165]:
function day1bs3(input)
    seen::Array{Int64, 1} = [0]
    freq = 0
    for i in Iterators.cycle(input)
        freq += i
        if freq ∈ seen
            return freq
        else
            push!(seen, freq)
        end
    end
end

day1bs3 (generic function with 1 method)

In [168]:
@time day1bs3(ld)

  3.430409 seconds (23 allocations: 3.001 MiB)


790

So not having a set is really slow searching!

In [170]:
using BenchmarkTools

In [174]:
@btime day1bs1(ld)  # with Set

  3.315 ms (34 allocations: 3.00 MiB)


790

In [175]:
@btime day1bs2(ld)  # with BitSet

  257.845 μs (10 allocations: 48.94 KiB)


790

In [177]:
@btime day1bs3(ld)  # With regular array [yuck!]

  3.281 s (19 allocations: 3.00 MiB)


790